### DCGAN для генерации лиц 

Задача: обучить DCGAN для генерации лиц и изучить латентное пространство.


Иллюстрация архитектуры ([источник](https://github.com/znxlwm/tensorflow-MNIST-GAN-DCGAN) иллюстрации).


In [0]:
%tensorflow_version 1.x
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import time
import os
from IPython import display
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import UpSampling2D, Conv2D, BatchNormalization, LeakyReLU, Conv2DTranspose, Reshape, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
init = RandomNormal(mean=0.0, stddev=0.02)

In [0]:

class DCGAN(tf.keras.Model):
    def __init__(self, image_size, output_path, num_channels=1, z_dim=100,
                 G_h_size=128, D_h_size=128):
        """
        image_size -- размер стороны квадратной картинки
        output_path -- путь для сохранения артефактов обучения. в корне -- картинки с разных итераций, 
        в папке model -- модель
        num_channels -- количество каналов изображения
        z_dim -- размерность латентного вектора
        G_h_size -- минимальный размер фильтров в сверточных слоях генератора
        D_h_size -- минимальный размер фильтров в сверточных слоях дискриминатора
        """
        super().__init__()
        self.image_size = image_size
        self.num_channels = num_channels
        self.z_dim = z_dim

        self.multiply = int(np.log2(self.image_size / 8)) # столько раз нужно применить апсемплинг или даунсемплинг
                                                          # чтобы из (4,4) получить (image_size/2, image_size/2) и наоборот
                                                
        self.output_path =  Path(output_path)
        (self.output_path / "model").mkdir(exist_ok=True)

        self.G_h_size = G_h_size
        self.D_h_size = D_h_size

        self.generator = self._build_generator()
        self.discriminator = self._build_discriminator()

        self.optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5, beta_2=0.999)
         
        self.gen_loss_hist = []
        self.disc_loss_hist = []
        self._vis_h = 5
        self._vis_w = 5
        self._vis_noise = np.random.normal(0, 1, (self._vis_h* self._vis_w, self.z_dim)).astype(np.float32)
        self.start_iteration = 0

    def discriminator_loss(self, real_output, fake_output):
        real_loss = cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    def generator_loss(self, fake_output):
        return cross_entropy(tf.ones_like(fake_output), fake_output)
    
    def _conv_bn_leaky(self, kernel_size, channels, stride=1):
        """
        Этот блок содержит Conv + BatchNorm + LeakyReLU

        При указании stride=2 -- уменьшит размер в два раза.
        """
        model = tf.keras.Sequential()
        model.add(Conv2D(channels,
                         kernel_size=kernel_size, padding="same",
                         use_bias=False, kernel_initializer=init,
                         strides=(stride, stride))) # use_bias=False, т.к. BatchNorm и так вычтет среднее
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        return model
        

    def _build_generator(self):
        """
        Генератор должен превращать вектор длины self.z_dim в 
        картинку image_size x image_size x num_channels

        """
        
        model = tf.keras.Sequential()
        # для начала сделаем вектор -- трехмерным тензором с помощью Reshape
        model.add(Reshape((1, 1, self.z_dim), input_shape=(self.z_dim,)))

        # Превратим его в тензор размера (4, 4, self.G_h_size * 2**self.multiply)
        model.add(Conv2DTranspose(self.G_h_size * 2**self.multiply,
                                  kernel_size=4, use_bias=False, 
                                  kernel_initializer=init))
        model.add(BatchNormalization())
        model.add(LeakyReLU())

        
        for i in range(self.multiply):
            model.add(UpSampling2D()) # увеличиваем картинку
            model.add(self._conv_bn_leaky(4, self.G_h_size * 2**self.multiply // 2**(i+1))) # уменьшаем количество фильтров в два раза
        
        assert model.output_shape[1:] == (self.image_size // 2, self.image_size // 2, self.D_h_size), f"{model.output_shape, self.D_h_size}"
        
        model.add(UpSampling2D())
        model.add(Conv2D(self.num_channels,
                         kernel_size=4, strides=(1, 1),
                         activation="tanh", padding="same", 
                         kernel_initializer=init))
        return model

    def _build_discriminator(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(
            input_shape=((self.image_size, self.image_size, self.num_channels))))
        model.add(self._conv_bn_leaky(kernel_size=4, 
                                      channels=self.D_h_size,
                                      stride=2,
                                      ))
        
        for i in range(self.multiply):
            model.add(self._conv_bn_leaky(kernel_size=4, 
                                          channels=self.D_h_size * (2 ** (i+1)),
                                          stride=2)) # количество фильтров увеличивается, размер уменьшается
        assert model.output_shape[1:] == (4, 4, self.D_h_size * 2**self.multiply), f"{model.output_shape}"
        model.add(Conv2D(1, kernel_size=4, kernel_initializer=init, use_bias=False)) # без активации !
        model.add(Flatten())
        return model
    
    @tf.function
    def train_step(self, images):
        noise = tf.random.normal([tf.cast(images.shape[0], tf.int32), self.z_dim])
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(noise, training=True)

            real_output = self.discriminator(images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            gen_loss = self.generator_loss(fake_output)
            disc_loss = self.discriminator_loss(real_output, fake_output)
            
        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        return gen_loss, disc_loss

    def save_imgs(self, epoch):
        """
        Сохранение промежуточных картинок на диск
        """
        gen_imgs = self.generator(self._vis_noise, training=False)
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(self._vis_h, self._vis_w, figsize=(6,6))
        cnt = 0
        for i in range(self._vis_h):
            for j in range(self._vis_w):
                if self.num_channels == 1:
                    axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                else:
                    axs[i, j].imshow(gen_imgs[cnt, :, :, :])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig(self.output_path / f"{epoch}.png")
        plt.show()
    
    def train(self, dataset, num_iters=2000, show_every=25):
        """
        Цикл обучения
        """
        start = time.time()
        iters = self.start_iteration
        for image_batch in dataset:
            print(".", end='')
            gen_loss, disc_loss = self.train_step(image_batch)
            
            self.disc_loss_hist.append(disc_loss.numpy())
            self.gen_loss_hist.append(gen_loss.numpy())    
            
            if iters % show_every == 0:
                display.clear_output(wait=True)
                plt.figure()
                plt.plot(self.disc_loss_hist, label="Discriminator loss")
                plt.plot(self.gen_loss_hist, label="Generator loss")
                plt.legend(loc="best")
                plt.figure()
                self.save_imgs(f"{iters}")
                self.save_weights(str(self.output_path / "model" / "dcgan_model"), save_format='tf')
                
                print(f"\n{iters}/{num_iters}")
                print(f'Time elapsed from start {time.time() - start} sec')
                
            iters += 1
            if iters > num_iters:
                print(f'Finished. Time elapsed from start {time.time() - start} sec')
                return
        


### Загрузка датасета

In [0]:
# ! pip install gdown
import gdown

url = 'https://drive.google.com/uc?id=0BxYys69jI14kYVM3aVhKS1VhRUk'
output = '/tmp/UTKFace.tar.gz'
gdown.download(url, output, quiet=False)
! tar -xzf /tmp/UTKFace.tar.gz -C /tmp/

### Создание генераторов данных

*Интенсивности картинок должны быть нормализованы от -1 до 1.*

In [0]:
BATCH_SIZE = 128
IMAGE_SIZE = 32
# os.listdir("/tmp/UTKFace/")

image_dir = Path("/tmp/UTKFace/")
filenames = list(map(lambda x: x.name, image_dir.glob('*.jpg')))
print(filenames[:3])

In [0]:
# формируем датафрейм

gender_mapping = {0: 'Male', 1: 'Female'}
correct_filenames, gender_labels, = [], []

for filename in filenames:
    if len(filename.split('_')) != 4:
        print(f"Bad filename {filename}")
        continue

    age, gender, race, _ = filename.split('_')
    correct_filenames.append(filename)
    gender_labels.append(gender)

data = {"img_name": correct_filenames, 
        "gender": gender_labels}

df = pd.DataFrame(data)
df.head()

In [0]:
# image_generator -- должен содержать необходимый генератор

# нормализуем из 0..255 в -1..1
def preprocess_input(image):
    x = (image - 127.5) / 127.5 
    return x

image_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
image_generator = image_gen.flow_from_dataframe(df, 
                                                directory=str(image_dir), 
                                                x_col='img_name', 
                                                y_col='gender', 
                                                batch_size=BATCH_SIZE, 
                                                shuffle=True, 
                                                class_mode=None, 
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE))


In [0]:
sample = next(image_generator)
assert sample.shape == (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), f"Размер батча должен быть: {(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3)}.  Получен {sample.shape}"
print("Shape test passed")
# если препроцессинг правильный, то картинка ниже имеет реалистичные цвета
plt.imshow((sample[0] + 1.) / 2)

### Обучение DCGAN

In [0]:
# Для того, чтобы сохранять прогресс и веса модели, будем использовать google drive -- так модель не потеряется
# и в случае отключения Colab - продолжить обучение.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
output = Path("/content/drive/My Drive/gan_utk_32")
output.mkdir(exist_ok=True)
(output / "model").mkdir(exist_ok=True)

gan = DCGAN(image_size=IMAGE_SIZE, num_channels=3, output_path=output, 
           z_dim=100, D_h_size=128, G_h_size=128)

In [0]:
# если при запуске обучения вы видите "WARNING:tensorflow:Entity <bound method DCGAN.train_step ...", 
# раскомментируйте строчку ниже, установите пакет и рестартните runtime
# это временный баг в tf из-за изменения версии gast
# ! pip install 'gast==0.2.2'

In [0]:
%%time 
images = list(output.glob("*.png"))
if images: # если папка не пуста, то продолжим обучение с последней итерации
    iters = list(map(lambda x: int(x.name.split(".")[0]), images))
    last_iter = sorted(iters)[-1]
    gan.start_iteration = last_iter
    print(f"Resuming model from {last_iter} iteration")
     
gan.train(image_generator, 10000, 50)

После того как модель обучена, можно посмотреть какие лица она научилась генерировать!

In [0]:
def generate_data(latent_vector, generator):
    """
    Для того чтобы сгенерировать объект нам нужен генератор и латентный вектор
    """
    gen_imgs = generator(latent_vector, training=False)
    gen_imgs = 0.5 * gen_imgs + 0.5
    return gen_imgs

In [0]:
v1 = tf.random.normal([1, 100]) # случайный вектор
print("Вектор: ", v1.numpy()[0, :10]) # распечатаем 10 первых элементов
_ = plt.imshow(generate_data(v1, gan.generator)[0]) # сгенерированное лицо 

### Поиск вектора улыбки

Найти “вектор улыбки” и доказать что он таковым является. 



In [0]:
# вспомогательный код
def generate_many(generator, n):
    vis_noise = np.random.normal(0, 1, (n, 100)).astype(np.float32)
    gen_imgs = generator(vis_noise, training=False)
    show_many(gen_imgs, "Generated images")
    return vis_noise

def show_many(images, title=""):
    w = h = int(np.sqrt(len(images)))
    images = (np.clip(images, -1, 1) + 1.) / 2. 
    
    fig, axs = plt.subplots(w, h, figsize=(w, h))
    if title != "":
        fig.suptitle(title)

    cnt = 0
    for i in range(h):
        for j in range(w):
            axs[i, j].imshow(images[cnt, :, :, :])
            axs[i, j].set_title(f"{cnt}")
            axs[i, j].axis('off')
            cnt += 1
    plt.subplots_adjust(wspace=.5)

### a) Интерполяция



In [0]:
def show_interpolation(v_1, v_2, generator, n=20):
    """
    Превращает v_1 в v_2 за n шагов, изображая 
    картинки соответствующие промежуточным векторам

    """
    fig, axs = plt.subplots(1, n, figsize=(n,1))
    for i, alpha in enumerate(np.linspace(0, 1, n)):
        curr_vec = v_1 * (1-alpha) + v_2 * alpha
        image = generate_data(curr_vec, gan.generator)[0]
        axs[i].imshow(image)
        axs[i].axis('off')

In [0]:
v1 = tf.random.normal([1, 100])
v2 = tf.random.normal([1, 100])

show_interpolation(v1, v2, gan.generator)

### b) Поиск вектора улыбки

Функция generate_many для того чтобы получить 100 сгенерированных изображений и соответствующие им вектора.

Лица пронумерованы и среди них есть лица
с улыбкой. Выберем около 10 лиц с улыбкой и 10 без,
запомним их номера. Затем посчитаем средний вектор
лица без улыбки и средний вектор лица с улыбкой. Разница
между ними и будет искомым вектором.

In [0]:
# поиск вектора улыбки

# получаем 100 сгенерированных изображений и соответствующие им вектора
gen_many = generate_many(gan.generator, 100)
gen_many

In [0]:
# 10 лиц с улыбкой 
smile_1 = #gen_many[i1]
smile_2 = #gen_many[i2]
smile_3 = #gen_many[i3]
smile_4 = #gen_many[i4]
smile_5 = #gen_many[i5]
smile_6 = #gen_many[i6]
smile_7 = #gen_many[i7]
smile_8 = #gen_many[i8]
smile_9 = #gen_many[i9]
smile_10 = #gen_many[i10]

# средний вектор лица с улыбкой 
smile_list = [smile_1, smile_2, smile_3, smile_4, smile_5, 
              smile_6, smile_7, smile_8, smile_9, smile_10]
mean_vec_smile = sum(smile_list)/len(smile_list)

# 10 лиц без улыбки 
not_smile_1 = #gen_many[i1]
not_smile_2 = #gen_many[i2]
not_smile_3 = #gen_many[i3]
not_smile_4 = #gen_many[i4]
not_smile_5 = #gen_many[15]
not_smile_6 = #gen_many[i6]
not_smile_7 = #gen_many[i7]
not_smile_8 = #gen_many[i8]
not_smile_9 = #gen_many[i9]
not_smile_10 = #gen_many[i10]

# средний вектор лица без улыбки
not_smile_list = [not_smile_1, not_smile_2, not_smile_3, not_smile_4, not_smile_5, 
                  not_smile_6, not_smile_7, not_smile_8, not_smile_9, not_smile_10]
mean_vec_not_smile = sum(not_smile_list)/len(not_smile_list)

# искомый вектор улыбки
vec_smile = mean_vec_smile - mean_vec_not_smile

In [0]:
# нейтральный человек ------> с улыбкой
not_smile_10 = tf.reshape(not_smile_10, [1, 100])
vec_smile = tf.reshape(vec_smile, [1, 100])
with_smile = not_smile_10 + vec_smile
show_interpolation(not_smile_10, with_smile, gan.generator)

In [0]:
# человек с улыбкой ------> нейтральный
smile_8 = tf.reshape(smile_8, [1, 100])
without_smile = smile_8 - vec_smile
show_interpolation(smile_8, without_smile, gan.generator)